## Preprocessing --GoDaddy

In [4]:
from constants import *
import pandas as pd

In [6]:
import pandas as pd

In [24]:
df_census= pd.read_csv(os.path.join(DATA_DIR, 'census_starter.csv'))

new_names_map= {}
for year in [2017,2018, 2019, 2020, 2021]:
    for attribute in ['pct_bb','pct_college', 'pct_foreign_born','pct_it_workers','median_hh_inc']:
        new_names_map[f"{attribute}_{str(year)}"]= f"{attribute}_{str(year+2)}"
        print("Renaming ",f"{attribute}_{str(year)}"," to",f"{attribute}_{str(year+2)}")
df_census= df_census.rename(columns=new_names_map)
df_census.head()
df_census.to_csv(os.path.join(DATA_DIR, 'census_starter_shifted.csv'))

Renaming  pct_bb_2017  to pct_bb_2019
Renaming  pct_college_2017  to pct_college_2019
Renaming  pct_foreign_born_2017  to pct_foreign_born_2019
Renaming  pct_it_workers_2017  to pct_it_workers_2019
Renaming  median_hh_inc_2017  to median_hh_inc_2019
Renaming  pct_bb_2018  to pct_bb_2020
Renaming  pct_college_2018  to pct_college_2020
Renaming  pct_foreign_born_2018  to pct_foreign_born_2020
Renaming  pct_it_workers_2018  to pct_it_workers_2020
Renaming  median_hh_inc_2018  to median_hh_inc_2020
Renaming  pct_bb_2019  to pct_bb_2021
Renaming  pct_college_2019  to pct_college_2021
Renaming  pct_foreign_born_2019  to pct_foreign_born_2021
Renaming  pct_it_workers_2019  to pct_it_workers_2021
Renaming  median_hh_inc_2019  to median_hh_inc_2021
Renaming  pct_bb_2020  to pct_bb_2022
Renaming  pct_college_2020  to pct_college_2022
Renaming  pct_foreign_born_2020  to pct_foreign_born_2022
Renaming  pct_it_workers_2020  to pct_it_workers_2022
Renaming  median_hh_inc_2020  to median_hh_inc_2022


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019-01-01
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019-01-01
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019-01-01
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019-01-01
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019-01-01


## Reformatting census data

In [57]:
years= [2019,2020,2021,2022,2023]
attributes= ['pct_bb','pct_college', 'pct_foreign_born','pct_it_workers','median_hh_inc']
##Adding the attributes to the dataframe df_train, for month take the value of the corresponding year and cfips
col_map = {}
col_map["year"] = []
for year in years:
    col_map["year"] += [year] * df_census.shape[0]
for attribute in attributes:
    col_map[attribute] = []
    for year in years:
        col_map[attribute] += list(df_census[attribute+ '_' + str(year)])
col_map["cfips"] = list(df_census["cfips"]) * len(years)
df_census_reshaped = pd.DataFrame(col_map)


# Fill in missing values with mean of the column for each cfips
for column in df_census_reshaped.columns:
    if column not in ['year', 'cfips']:
        df_census_reshaped[column] = df_census_reshaped.groupby('cfips')[column].apply(lambda x: x.fillna(x.mean()))
df_census_reshaped.to_csv(os.path.join(DATA_DIR,"census_ae.csv"), index=False)

In [32]:
# len(df_census_reshaped["cfips"].unique())

3142

## Train test splitting of the data df_census_reshaped

In [58]:
from sklearn.model_selection import train_test_split
## 80/20 split of df_census_reshaped and save to train_census_ae and train_census_ae_val
df_train_census_ae, df_train_census_ae_val = train_test_split(df_census_reshaped, test_size=0.2, random_state=42)
df_train_census_ae.to_csv(os.path.join(DATA_DIR, 'census_ae_train.csv'), index=False)
df_train_census_ae_val.to_csv(os.path.join(DATA_DIR, 'census_ae_val.csv'), index=False)




In [59]:

df_train= pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
df_train['year']=pd.to_datetime(df_train['first_day_of_month'].str.split("-", expand=True)[0])
df_train.head()


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019-01-01
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019-01-01
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019-01-01
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019-01-01
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019-01-01


In [60]:
df_train["microbusiness_density"].mean()
df_train["microbusiness_density"].std()

4.991086802428146

In [54]:
df_train.describe()

,cfips,microbusiness_density,active
count,122265.000000,122265.000000,1.222650e+05
mean,30376.037640,3.817671,6.442858e+03
std,15143.508721,4.991087,3.304001e+04
min,1001.000000,0.000000,0.000000e+00
25%,18177.000000,1.639344,1.450000e+02
50%,29173.000000,2.586543,4.880000e+02
75%,45077.000000,4.519231,2.124000e+03
max,56045.000000,284.340030,1.167744e+06


In [66]:

df_census_reshaped["year"]=pd.to_datetime(df_census_reshaped["year"], format='%Y')

In [68]:
df_train_update= pd.merge(df_train, df_census_reshaped, how='left', on=['year', 'cfips'])

In [69]:
df

,cfips,microbusiness_density,active,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc
count,122265.000000,122265.000000,1.222650e+05,122265.000000,122265.000000,122265.000000,122265.000000,122265.000000
mean,30376.037640,3.817671,6.442858e+03,74.686369,14.220153,4.744932,1.355765,52830.752165
std,15143.508721,4.991087,3.304001e+04,9.374487,5.685812,5.711121,0.960021,14128.755489
min,1001.000000,0.000000,0.000000e+00,24.500000,0.000000,0.000000,0.000000,19264.000000
25%,18177.000000,1.639344,1.450000e+02,69.300000,10.100000,1.400000,0.700000,43507.000000
50%,29173.000000,2.586543,4.880000e+02,75.800000,13.200000,2.700000,1.200000,51094.000000
75%,45077.000000,4.519231,2.124000e+03,81.200000,17.300000,5.700000,1.800000,59230.000000
max,56045.000000,284.340030,1.167744e+06,97.100000,48.000000,54.000000,17.400000,147111.000000
